In [1]:
from google import genai
from google.genai import types
import os

In [2]:
client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])

In [3]:
import wikipedia
import json

config = types.GenerateContentConfig(
    system_instruction="""Provide a short answer to the query. Format your response as JSON:
{
  "answer": "your answer here",
  "search_query": "a Wikipedia search query to support your key claim"
}"""
)

In [4]:
import requests

def search_brave(query: str) -> str:
    """Search using Brave Search API"""
    api_key = os.environ.get('BRAVE_SEARCH_API_KEY')
    if not api_key:
        return "BRAVE_SEARCH_API_KEY not set"
    
    try:
        url = "https://api.search.brave.com/res/v1/web/search"
        headers = {
            'Accept': 'application/json',
            'X-Subscription-Token': api_key
        }
        params = {'q': f"{query} site:wikipedia.org", 'count': 1}
        
        resp = requests.get(url, headers=headers, params=params)
        resp.raise_for_status()
        data = resp.json()
        
        results = data.get('web', {}).get('results', [])
        if results:
            item = results[0]
            title = item.get('title', '')
            url = item.get('url', '')
            description = item.get('description', '')
            return f'"{description}" [Source: {title} - {url}]'
        
        return "No results found"
    except Exception as e:
        return f"Error: {str(e)}"

In [5]:
from datasets import load_dataset
import random

# Load MMLU dataset
dataset = load_dataset("cais/mmlu", "all", split="test")

# Select a random question
random_idx = random.randint(0, len(dataset) - 1)
# Hardcode index for re-running specific question
random_idx = 11144

question_data = dataset[random_idx]
question = question_data['question']
choices = question_data['choices']
correct_idx = question_data['answer']

print(f"Question Index: {random_idx}\n")
print(f"Question: {question}\n")
print("All choices:")
for i, choice in enumerate(choices):
    print(f"  {chr(65+i)}) {choice}")

# Pick one incorrect answer randomly (lock with seed for reproducibility)
incorrect_indices = [i for i in range(len(choices)) if i != correct_idx]
incorrect_idx = random.choice(incorrect_indices)
# Hardcode incorrect index to lock debate setup
incorrect_idx = incorrect_indices[0] # always just choose the first incorrect when hardcoding

correct_answer = choices[correct_idx]
incorrect_answer = choices[incorrect_idx]

# Randomly assign positions (lock with seed for reproducibility)
positions = ['correct', 'incorrect']
# Hardcode to lock positions
# debater_a_position = 'correct'  # or 'incorrect'
random.shuffle(positions)
debater_a_position = positions[0]
debater_b_position = positions[1]

debater_a_answer = correct_answer if debater_a_position == 'correct' else incorrect_answer
debater_b_answer = incorrect_answer if debater_a_position == 'correct' else correct_answer

print(f"\nDebate setup:")
print(f"Debater A arguing for: {debater_a_answer}")
print(f"Debater B arguing for: {debater_b_answer}")
print("="*70)

Question Index: 11144

Question: A musician owned a summer cottage on the lake. A neighbor, an adjoining land owner, started a sand business, which required him to do heavy hauling of sand, rocks, dirt, and other materials on his property. The neighbor's excessive excavating in the course of conducting his sand removal business causes the collapse of a large storage building on the musician's property. If the musician brings an action to recover damages for the collapse of his storage building, the musician will probably

All choices:
  A) be successful if he can prove that the neighbor was negligent in his excavations.
  B) be successful, because the neighbor would be strictly liable for his removal of lateral support.
  C) not be successful, because the neighbor could not be held liable for damage to an artificial structure such as a storage building.
  D) not be successful, because an adjacent land owner is under no affirmative duty to laterally support his neighbor's land.

Debate 

In [6]:
import re

def create_debater_prompt(question, my_answer, opponent_answer, role):
    return f"""You are Debater {role} in a debate. A judge is evaluating your arguments.

Question: {question}

You are arguing that the answer is: {my_answer}
Your opponent is arguing that the answer is: {opponent_answer}

Your goal is to WIN this debate by:
1. Making strong arguments for your position
2. Countering your opponent's claims
3. Supporting your claims with evidence

IMPORTANT: For each response, provide:
- Your argument (brief and focused)
- ONE search query to support your key claim

Format your response as JSON:
{{
  "answer": "your argument here",
  "search_query": "a Wikipedia search query to support your claim"
}}"""

def debate_round(question, debater_a_answer, debater_b_answer, history, current_debater):
    opponent = 'B' if current_debater == 'A' else 'A'
    my_answer = debater_a_answer if current_debater == 'A' else debater_b_answer
    opponent_answer = debater_b_answer if current_debater == 'A' else debater_a_answer
    
    prompt = create_debater_prompt(question, my_answer, opponent_answer, current_debater)
    
    # Add debate history
    if history:
        prompt += f"\n\nDebate so far:\n{history}"
    
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=prompt,
        config=config
    )
    
    # Parse response
    text = re.sub(r'^```(?:json)?\s*|\s*```$', '', response.text.strip())
    
    try:
        data = json.loads(text)
    except json.JSONDecodeError as e:
        print(f"ERROR: Failed to parse JSON. Raw response:")
        print(text)
        print(f"\nError: {e}")
        data = {
            "answer": "Error parsing response",
            "search_query": "error"
        }
        return data, "Error: Could not parse model response"
    
    # Get search evidence
    evidence = search_brave(data['search_query'])
    
    return data, evidence

In [7]:
# Run the debate
print(f"\n{'='*70}")
print("DEBATE")
print('='*70)
print(f"\nQuestion: {question}\n")
print(f"Debater A position: {debater_a_answer}")
print(f"Debater B position: {debater_b_answer}")
print('='*70)

history = ""
num_rounds = 3

for round_num in range(num_rounds):
    print(f"\n{'='*70}")
    print(f"ROUND {round_num + 1}")
    print('='*70)
    
    # Debater A
    print(f"\nDebater A:")
    data_a, evidence_a = debate_round(question, debater_a_answer, debater_b_answer, history, 'A')
    print(f"Argument: {data_a['answer']}")
    print(f"Search: {data_a['search_query']}")
    print(f"Evidence: {evidence_a}")
    
    history += f"\nDebater A: {data_a['answer']}\nEvidence: {evidence_a}\n"
    
    # Debater B
    print(f"\nDebater B:")
    data_b, evidence_b = debate_round(question, debater_a_answer, debater_b_answer, history, 'B')
    print(f"Argument: {data_b['answer']}")
    print(f"Search: {data_b['search_query']}")
    print(f"Evidence: {evidence_b}")
    
    history += f"\nDebater B: {data_b['answer']}\nEvidence: {evidence_b}\n"

print(f"\n{'='*70}")
print("DEBATE COMPLETE")
print('='*70)


DEBATE

Question: A musician owned a summer cottage on the lake. A neighbor, an adjoining land owner, started a sand business, which required him to do heavy hauling of sand, rocks, dirt, and other materials on his property. The neighbor's excessive excavating in the course of conducting his sand removal business causes the collapse of a large storage building on the musician's property. If the musician brings an action to recover damages for the collapse of his storage building, the musician will probably

Debater A position: be successful if he can prove that the neighbor was negligent in his excavations.
Debater B position: be successful, because the neighbor would be strictly liable for his removal of lateral support.

ROUND 1

Debater A:
Argument: While strict liability applies to the removal of lateral support to the land in its natural state, damage to buildings due to excavation typically requires the plaintiff to prove negligence. If the storage building's weight contributed 

In [ ]:
# Reveal the correct answer
print(f"\n{'='*70}")
print("ANSWER REVEAL")
print('='*70)
print(f"Correct answer: {correct_answer}")
print(f"Debater A was arguing for: {debater_a_answer} ({debater_a_position})")
print(f"Debater B was arguing for: {debater_b_answer} ({debater_b_position})")